In [1]:
import os
import pickle
import keras
import numpy as np
import sys
import argparse
import tensorflow as tf
import keras.backend as K

from keras.datasets import cifar10
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential, Model
from keras.layers import Dense, BatchNormalization, Input, Activation, Conv2D, MaxPool2D, AveragePooling2D, Dropout, Flatten, Layer, ZeroPadding2D, Add, Lambda
from keras.wrappers.scikit_learn import KerasClassifier
from keras.callbacks import LearningRateScheduler, EarlyStopping, ModelCheckpoint, Callback
from keras.backend.tensorflow_backend import set_session
from keras.losses import categorical_crossentropy
from keras.optimizers import SGD, Adam, RMSprop
from keras.initializers import glorot_uniform
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2

config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.3
set_session(tf.Session(config=config))
IMG_SHAPE = (32, 32, 3)
WEIGHT_DECAY = 0.0005
FEATURES_DIM = 128


from keras.models import Model
from keras.layers import Input, Add, Activation, Dropout, Flatten, Dense
from keras.layers.convolutional import Convolution2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras import backend as K
from keras.regularizers import l2

weight_decay = 0.0005

Using TensorFlow backend.


In [15]:
def conv_factory(input, filters, ksize, padding = 'same', pooling = None, pool_size = 3, activation = 'relu', dropout = 0.5):

    def frac_max_pool(x):
        return tf.nn.fractional_max_pool(x, [1.0, 1.44, 1.44, 1.0])[0]

    inner = Conv2D(filters, kernel_size = ksize, padding = padding, kernel_regularizer = l2(WEIGHT_DECAY))(input)
    if pooling == 'mp':
        inner = MaxPool2D(pool_size = pool_size, strides = 2)(inner)
    elif pooling == 'fmp':
        inner = Lambda(frac_max_pool)(inner)
    else:
        pass

    inner = BatchNormalization()(inner)
    inner = Activation(activation=activation)(inner)
    inner = Dropout(dropout)(inner)
    return inner

def base_model(pooling = 'mp', activation = 'relu', summary = True, optimizer = 'adam'):
    inp = Input(shape=(IMG_SHAPE))
    inner = conv_factory(inp, 32, 5, pooling= None, activation = activation, dropout=0.0)
    inner = conv_factory(inner, 32, 5, pooling = pooling, activation = activation, dropout=0.0)

    inner = conv_factory(inner, 64, 5, pooling= None, activation = activation, dropout=0.25)
    inner = conv_factory(inner, 64, 5, pooling = pooling, activation = activation, dropout=0.25)

    inner = conv_factory(inner, 128, 5, pooling= None, activation = activation, dropout=0.5)
    inner = conv_factory(inner, 128, 5, pooling = pooling, activation = activation, dropout=0.5)

#     inner = conv_factory(inner, 256, 3, pooling= None, activation = activation, dropout=0.5)
    # inner = conv_factory(inner, 256, 3, pooling = pooling, activation = activation, dropout=0.5)

    embedding = Flatten(name='embedding')(inner)
    out = Dense(256, activation=activation)(embedding)
    out = Dense(512, activation=activation)(out)
    out = Dropout(rate = 0.5)(out)
    out = Dense(10, name='output')(out)
    out = Activation(activation='softmax')(out)

    model = Model(inputs=inp, outputs=out)
    model.compile(loss='categorical_crossentropy',
                  optimizer = optimizer, metrics=['accuracy'])
    if summary:
        model.summary()
    return model

In [3]:
!ls

Bai_tap_GK.docx  cifar10.py		    week 1  week 6	  WRN-28-9.png
best_weights.h5  cnnfmp_4212010_85.81.h5    week 2  week 7
center.png	 GT_ML_Huan_sau_truong.pdf  week 3  week 8
center_wrn.png	 __pycache__		    week 4  week 9
cifar10.ipynb	 README.md		    week5   WRN-28-8.png


In [16]:
model = base_model('fmp')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_39 (Conv2D)           (None, 32, 32, 32)        2432      
_________________________________________________________________
batch_normalization_39 (Batc (None, 32, 32, 32)        128       
_________________________________________________________________
activation_45 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
dropout_45 (Dropout)         (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_40 (Conv2D)           (None, 32, 32, 32)        25632     
_________________________________________________________________
lambda_19 (Lambda)           (None, 22, 22, 32)        0         
__________

In [18]:
model.load_weights("cnnfmp_4212010_85.81.h5")

In [19]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)

In [28]:
score = model.evaluate(X_test, y_test, batch_size=16)

 9984/10000 [============================>.] - ETA: 0s

InvalidArgumentError: Incompatible shapes: [16] vs. [32]
	 [[Node: metrics_6/acc/Equal = Equal[T=DT_INT64, _device="/job:localhost/replica:0/task:0/device:GPU:0"](metrics_6/acc/ArgMax, metrics_6/acc/ArgMax_1)]]
	 [[Node: loss_6/add_5/_1245 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_554_loss_6/add_5", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'metrics_6/acc/Equal', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.5/dist-packages/traitlets/config/application.py", line 658, in launch_inst

In [30]:
acc =  0
num_it = X_test.shape[0]//16 + 1
for i in range(num_it):
    left = i * 32
    right = i * 32 + 32
    if right >= X_test.shape[0]:
        right = X_test.shape[0]
        left = right- 32
    acc += model.evaluate(X_test[left:right], y_test[left:right], 32)[1]

print(np.mean(acc))

32/32 [==============================] - 0s 3ms/step


32/32 [==============================] - 0s 3ms/step


32/32 [==============================] - 0s 3ms/step


32/32 [==============================] - 0s 3ms/step


32/32 [==============================] - 0s 3ms/step
543.84375


In [31]:
543.84375 / (num_it)

['loss', 'acc']